# Import

In [1]:
import numpy as np
import pandas as pd

# Data preporation

In [2]:
# Load data of previous day
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
6560,2020-05-27,Челябинская область,Chelyabinsk region,74.0,70.0,0.0,64.0,2286.0,13.0,988.0
6561,2020-05-27,Чеченская Республика,Chechen Republic,95.0,19.0,0.0,8.0,1152.0,13.0,698.0
6562,2020-05-27,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,1.0,63.0,1.0,37.0
6563,2020-05-27,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,51.0,0.0,30.0,2202.0,6.0,788.0
6564,2020-05-27,Ярославская область,Yaroslavl region,76.0,82.0,0.0,47.0,2808.0,13.0,616.0


In [3]:
# Read update data
# Result of data_parsing_SCRF.ipynb
file_name = f'covid19-russia-cases-upd-scrf.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,173497,71251,2254,2020-05-28,2054.0,71.0,3793.0
30,Московская область,36730,6843,380,2020-05-28,774.0,21.0,403.0
64,Санкт-Петербург,14846,4783,167,2020-05-28,383.0,9.0,325.0
33,Нижегородская область,8963,3244,84,2020-05-28,228.0,6.0,339.0
67,Свердловская область,4693,1827,23,2020-05-28,268.0,2.0,127.0


In [4]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

# Updating

In [5]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [6]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
6640,2020-05-28,Тюменская область,Tyumen region,72.0,30.0,0.0,63.0,1562.0,7.0,757.0
6641,2020-05-28,Удмуртская Республика,Udmurt republic,18.0,24.0,0.0,14.0,583.0,12.0,381.0
6642,2020-05-28,Ульяновская область,Ulyanovsk region,73.0,75.0,0.0,28.0,2196.0,7.0,614.0
6643,2020-05-28,Хабаровский край,Khabarovsk region,27.0,63.0,1.0,40.0,1890.0,11.0,843.0
6644,2020-05-28,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,104.0,0.0,47.0,2028.0,17.0,716.0
6645,2020-05-28,Челябинская область,Chelyabinsk region,74.0,82.0,1.0,69.0,2368.0,14.0,1057.0
6646,2020-05-28,Чеченская Республика,Chechen Republic,95.0,22.0,0.0,47.0,1174.0,13.0,745.0
6647,2020-05-28,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,3.0,0.0,3.0,66.0,1.0,40.0
6648,2020-05-28,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,47.0,1.0,68.0,2249.0,7.0,856.0
6649,2020-05-28,Ярославская область,Yaroslavl region,76.0,84.0,0.0,37.0,2892.0,13.0,653.0


In [7]:
# Negative values fix
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(diff_c, diff_d, diff_r, last['Region/City'])
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [8]:
# Manual fixes if needed
# rus_df.loc[287, 'Confirmed'] = 5
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0

# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [9]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 379051.0 | By day: 379051.0
Russia Deaths: 4142.0 | By day: 4142.0
Russia Recovered: 150993.0 | By day: 150993.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [10]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df.to_csv(file_name, index=False)